In [1]:
import numpy as np

[Activation functions](https://github.com/Kulbear/deep-learning-nano-foundation/wiki/ReLU-and-Softmax-Activation-Functions)

In [2]:
#Sigmoid
def sigmoid(x):
	  return 1 / (1 + np.exp(-x))
    
#ReLU
def ReLU(x):
    return x*(x>0)

#Softmax
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

    
    

In [3]:
import os
from sklearn.datasets import fetch_mldata

# Fetch MNIST dataset and create a local copy.
if os.path.exists('mnist.npz'):
    with np.load('mnist.npz', 'r') as data:
        X = data['X']
        y = data['y']
else:
    mnist = fetch_mldata("mnist-original")
    X, y = mnist.data / 255.0, mnist.target
    np.savez('mnist.npz', X=X, y=y)

In [7]:
print(np.shape(X))

(70000, 784)
